<a href="https://colab.research.google.com/github/NLP-END3/Session5/blob/main/Session5_END_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.datasets import IMDB,YelpReviewFull,YahooAnswers

In [2]:
help(IMDB), help(YelpReviewFull), help(YahooAnswers)

Help on function IMDB in module torchtext.datasets.imdb:

IMDB(root='.data', split=('train', 'test'))
    IMDB dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 25000
    
        test: 25000
    
    
    Number of classes
        2
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')

Help on function YelpReviewFull in module torchtext.datasets.yelpreviewfull:

YelpReviewFull(root='.data', split=('train', 'test'))
    YelpReviewFull dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 650000
    
        test: 50000
    
    
    Number of classes
        5
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Ca

(None, None, None)

In [3]:
data_set = input('What is your choice of dataset? : IMDB, YelpReviewFull, YahooAnswers')

What is your choice of dataset? : IMDB, YelpReviewFull, YahooAnswersIMDB


In [4]:
print(f'Chosen dataset : {data_set}')

Chosen dataset : IMDB


In [5]:
train_iter_IMDB = IMDB(split = 'train')
#train_iter_YelpReviewFull = YelpReviewFull(split = 'train')
#train_iter_YahooAnswers = YahooAnswers(split = 'train')

100%|██████████| 84.1M/84.1M [00:05<00:00, 14.2MB/s]


## Selecting the right dataset

In [6]:
data_sets = {'IMDB':train_iter_IMDB} #, 'YelpReviewFull': train_iter_YelpReviewFull, 'YahooAnswers': train_iter_YahooAnswers}
train_iter = data_sets[data_set]

## Exploring the dataset

In [7]:
len(train_iter), type(train_iter), next(iter(train_iter))

(25000,
 torchtext.data.datasets_utils._RawTextIterableDataset,
 ('neg',
  'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered 

In [8]:
#for (line_number, (label,text)) in enumerate(train_iter):
#  if line_number < 5:
#    print(label,text)
#    if label == 'neg':
#      neg_count += 1
#print(f'Number of negative reviews : {neg_count}')

## NUmber of positive and Negative examples

In [9]:
train_iter = data_sets[data_set]
neg_count = 0
pos_count = 0

for (line_number, (label,text)) in enumerate(train_iter):
  if line_number < 5:
    print(label, text)
  if label == 'neg':
    neg_count += 1
  else:
    pos_count += 1

print(f'Number of Positive reviews : {pos_count}')
print(f'Number of Negative reviews : {neg_count}')

neg "I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same 

## Tokenization & Vocabulary

In [10]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [11]:
tokenizer = get_tokenizer('basic_english')

In [12]:
tokenizer("I am an Indian since 1990")

['i', 'am', 'an', 'indian', 'since', '1990']

In [13]:
train_iter = IMDB(split = 'train')

In [14]:
len(train_iter)

25000

In [15]:
def yield_tokens(data_iter):
  for _,text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [16]:
len(vocab) # vocab size is 100683

100683

## Text and Label Pipeline

In [17]:
text_pipeline = lambda x : vocab(tokenizer(x))
label_pipeline = lambda x : 1 if x == 'pos' else 0

In [18]:
text_pipeline("the movie is great")

[1, 20, 9, 92]

In [19]:
label_pipeline('neg'), label_pipeline('pos')

(0, 1)

## Collate_fn

In [20]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)  

In [21]:
from torch.utils.data import DataLoader
train_iter = IMDB(split='train')
dataloader = DataLoader(train_iter, batch_size = 8, shuffle = False, collate_fn = collate_batch)

## Neural Network Model

In [22]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [23]:
train_iter = IMDB(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
print(f'Vocabulary Size : {vocab_size}')
print(f'Embedding Size : {emsize}')
print(f'Number of Classes : {num_class}')

Vocabulary Size : 100683
Embedding Size : 64
Number of Classes : 2


In [24]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Running the model

In [25]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time: 10.34s | valid accuracy    0.653 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  9.94s | valid accuracy    0.734 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time: 10.02s | valid accuracy    0.839 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time: 10.09s | valid accuracy    0.854 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  9.97s | valid accuracy    0.865 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time: 10.01s |

## Testing Accuracy

In [26]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.867
